# Human-in-the-loop

Showcases how to involve human user in the loop.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.workflow import Workflow
from vanilla_aiagents.team import Team
from vanilla_aiagents.user import User
from vanilla_aiagents.llm import AzureOpenAILLM
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.team").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.llm").setLevel(logging.DEBUG)

In [ ]:
agent1 = Agent(id="agent1", llm=llm, description="Call this agent for general purpose questions", system_message = """You are an AI assistant
    Your task is to help the user with their questions.
    Always respond with the best answer you can generate.
    If you don't know the answer, respond with "I don't know".
    Always be polite and helpful.
    """)


In [ ]:
# User involvement here. In this case, mode is unattended and we'll provide user input manually
user = User(id="user", mode="unattended")

In [ ]:
# Assemble the team and workflow
flow = Team(id="team", description="", members=[agent1, user], llm=llm, stop_callback=lambda conv: len(conv.messages) > 6)
workflow = Workflow(askable=flow)

In [ ]:
# First round
workflow.run("Hi! Can you help me with capital cities?")
workflow.conversation.messages

In [ ]:
# Second round, manually providing user input
workflow.run("Which is the capital of France?")
workflow.conversation.messages